<a href="https://colab.research.google.com/github/johnmatsson/32feet/blob/main/Commentsmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')


!ls /content/gdrive/MyDrive/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 5089.rar
 adrenoabs.key
 adrenoabs.key.der
 adrenoabs.p12
 adrenoabs.pub
 Adreno.asc
 aktivaon2.png
 aktivationfx.png
 Ara
'Backup of Dropbox (adrenochromie@gmail.com)'
 Bytes_starttider_sektionsvis_m_enkelstatistik.xlsx
 classes.js
'Colab Notebooks'
 commentsprepared.txt
 Datasets
 DataWrangler-0.1.tar.gz
 deploy.prototxt
'DjVu Viewer and Reader (1).png'
'DjVu Viewer and Reader.png'
 DS14C89AM.pdf
 ecg5000
 ecg5000_98.6_model.h5
 ecg5000_98.6_model.keras
 File_161b05ea998_9f2dbcb7-65b4-4ab6-b315-61ad2c8179f3_2.odt.gdoc
 forwardNN.png
'Google Foto'
 image.JPG
 IMG_0001.JPG
 IMG_0004.JPG
 IMG_0008.JPG
 IMG_0015.JPG
 IMG_0016.JPG
 IMG_0020.JPG
 IMG_0031.JPG
'instruktion-for-polisanmalan (1).docx'
 Jobbet
'Landstinget i Kalmar.png'
'login homerun'
'login homerun.gdoc'
 Mechanical_Filter_Tester_KK6IL.gif
 mobilenet_iter_73000.caffemodel
 photo_2022-11-21_15-52

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.1 MB/s eta 0:00:00


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

drive.mount('/content/gdrive')

# Specify the path to your file
file_path = '/content/gdrive/MyDrive/commentsprepared.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read().splitlines()

# Create a DataFrame from the data
df = pd.DataFrame({'column_name': data})

# Tokenization and Language Label Encoding
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# Limit the number of tokens per sentence (adjust this value as needed)
max_seq_length = 128
df['sentence'] = df['column_name'].apply(lambda x: x[:max_seq_length])

X = df['sentence'].values.tolist()  # Convert sentences to a list of strings
X = tokenizer(X, padding=True, truncation=True, return_tensors="tf")
y_language = [0] * len(df)  # All sentences are assumed to be in Swedish, so label them as 0.

# Convert the tokenized data to numpy arrays
X_input_ids = np.array(X['input_ids'])
y_language = np.array(y_language)

# Split the data into training and testing sets
X_train, X_test, y_language_train, y_language_test = train_test_split(X_input_ids, y_language, test_size=0.2, random_state=42, shuffle=False)

# Reduce batch size (adjust this value as needed)
batch_size = 8

# Load a pretrained BERT model
model_name = "bert-base-multilingual-cased"
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=1)

# Define a custom loss function for binary classification (language detection)
def custom_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = y_pred[:, 0]
    return tf.keras.losses.binary_crossentropy(y_true, y_pred)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss=custom_loss)

# Train the model with the adjusted batch size
model.fit(X_train, y_language_train, validation_data=(X_test, y_language_test), epochs=2, batch_size=batch_size)

# Print a message to indicate completion
print("Completed!")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
21/21 [==============================] - 201s 8s/step - loss: 6.6896e-05 - val_loss: 0.0000e+00
Epoch 2/2
21/21 [==============================] - 154s 7s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Completed!


In [29]:


import numpy as np

# Assuming you already have the model loaded and the dataset prepared
# model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=1)
# X_train, y_language_train, and X_test, y_language_test are prepared as in your previous code

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss=custom_loss)

# Evaluate the model on the training dataset
train_loss = model.evaluate(X_train, y_language_train, verbose=1)

# Evaluate the model on the test dataset
test_loss = model.evaluate(X_test, y_language_test, verbose=1)

# Print the results
print("Training Loss:", train_loss)
print("Test Loss:", test_loss)



2/2 [==============================] - 8s 2s/step - loss: 0.0497
Training Loss: 0.056813135743141174
Test Loss: 0.04970669746398926


In [ ]:
model_path = '/content/gdrive/MyDrive/lang_model'

# Save the model
model.save(model_path)

# Print a message to indicate that the model has been saved
print("Model saved at:", model_path)

In [30]:
# Assuming you've already trained the model as before

# Re-split the data
X_train, X_test, y_language_train, y_language_test = train_test_split(X_input_ids, y_language, test_size=0.2, random_state=32, shuffle=True)

# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_language_test, verbose=1)

# Print the test loss
print("Test Loss:", test_loss)


2/2 [==============================] - 13s 3s/step - loss: 0.0480
Test Loss: 0.047986552119255066
